<a href="https://colab.research.google.com/github/williameclee/trade-cumulus-inversion/blob/main/invtci.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trade Cumulus Inversion

## Loading of necessary libraries
The required libraries include: `numpy`, `scipy`, `matplotlib`, and custom libraries `inv` and `updateboundary`.

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from inv import *
import updateboundary

## Speficication of experiment parameters

In [ ]:
# Experiment dimensions
K = 61
J = 81
Kt = K + 2
Jt = 2 * J + 1

# Physical parameters
R = 287
cp = 1004
kappa = R / cp
gr = 9.81
Ae = 6.371e06
Omega = 7.292e-05
rom = Ae * Omega

## Initialising paremeters for experiment 2

In [ ]:
thetat = 320  # theta (kelvin) at model top
thetab = 300  # theta (kelvin) at model bottom
sigma0 = 45 * 100  # sigma0 (Pa/K)
pb = 1000 * 100  # pressure (Pa) at model bottom

Dp = 58 * 100  # pressure (Pa) change across inversion
theta10 = 303  # mean theta (kelvin) in inversion
Dtheta = 4  # dtheta (kelvin) of inversion
thetac = 3  # thetac (kelvin) of inversion

phis = -30  # southern extent of inversion (degrees)
phin = 30  # northern extent of inversion (degrees)
phic = 0  # latitude (degrees) of inversion center

pt = pb - sigma0 * (thetat - thetab) + Dp

In [ ]:
alpha = (pb - pt) / pb
beta = thetab / (thetat - thetab)
cs = alpha * R * (thetat - thetab)
eps = 4 * rom**2 / cs

## Initialising coordinates

In [ ]:
# Horizontal grid
Y = np.linspace(-1, 1, Jt)  # Potential latitude (S) in the paper
DY = Y[1] - Y[0]
phi = np.rad2deg(np.arcsin(Y))

# Vertical grid
Z = np.linspace(0, 1, K)
DZ = Z[1] - Z[0]
th = np.linspace(thetab, thetat, K).T
Dth = th[1] - th[0]
th_pad = np.concatenate((np.array([thetab - Dth]), th, np.array([thetat + Dth])))

YY, Th_pad = np.meshgrid(Y, th_pad, indexing="ij")
phii = np.meshgrid(phi, th_pad, indexing="ij")[0]

fac1 = eps * DY * Y
fac2 = 2 * DY * DZ**2
f1 = fac1 * (1 - Y**2)

## Initialising variables for experiment 2

In [ ]:
# Theta
phi_N = (phi <= phin) & (phi > phic)
phi_S = (phi >= phis) & (phi < phic)
phi_C = phi == phic
th1 = np.full((Jt), theta10)

th1[phi_N] = theta10 + Dtheta * fexp((phin - phi[phi_N]) / (phin - phic))
th1[phi_S] = theta10 + Dtheta * fexp((phi[phi_S] - phis) / (phic - phis))
th1[phi_C] = theta10 + Dtheta
th2 = th1 + thetac

del phi_N, phi_S, phi_C

#  Pressure
th1_grid = np.meshgrid(th1, th_pad, indexing="ij")[0]
th2_grid = np.meshgrid(th2, th_pad, indexing="ij")[0]
th_T = Th_pad >= th2_grid
th_C = (Th_pad > th1_grid) & (Th_pad < th2_grid)
th_B = Th_pad <= th1_grid
Pres = pb - sigma0 * (Th_pad - thetab)
Pres[th_T] = Pres[th_T] + Dp
Pres[th_C] = Pres[th_C] + Dp * fexp((Th_pad[th_C] - th1_grid[th_C]) / thetac)
Pres = Pres / pb
pref = np.zeros([1, Kt])
for k in range(Kt):
    pref[0, k] = pmean(Pres[:, k], Y)
Gamma = Pres ** (kappa - 1)

In [ ]:
# compute initial m field at even points
# the M (Montgomery potential) is normalised by a factor of cs
X = np.zeros((Jt, Kt))  # (-jc+1:jc-1,-1:kcp)
# xtmp = np.zeros(K + 1)  # (0:kc)
# ftmp = np.zeros(K + 1)  # (0:kc)

X[1:-1:2, 1] = cp * thetab / cs
for j in range(1, Jt - 1, 2):
    Mp = Pres[j, 1:-1] ** kappa / (kappa * alpha)
    # xtmp = intdde(K, DZ, x[j, 0], Mp)
    X[j, 1:-1] = intdde(Mp, X[j, 0], DZ)
X[1:-1:2, 0] = X[1:-1:2, 2] - 2 * DZ * X[1:-1:2, 1] / beta
X[1:-1:2, -1] = X[1:-1:2, -3] + 2 * DZ * (1 - alpha) ** kappa / (kappa * alpha)

# initialise x (i.e., sin(phi)) at odd points
X[::2, :] = np.tile(Y[::2], (Kt, 1)).T

## Compute Sigma

In [ ]:
tinc = 0.1
sigma = np.ones((Jt, Kt)) 

# Calculate sigma values
for j in range(Jt):
    for k in range(1,Kt):
        if th_C[j, k] != 1:
            continue
        tp = th[k] + tinc
        tm = th[k] - tinc
        fp = (tp - th1[j]) / (th2[j] - th1[j])
        fm = (tm - th1[j]) / (th2[j] - th1[j])
        dfdt = (fp - fm) / (2 * tinc)
        sigma[j, k] = (sigma0 - Dp * dfdt) / sigma0

del th1_grid, th2_grid, th_T, th_C

In [ ]:
X = updateBoundary_M(X, Y, DZ, eps, alpha, beta, kappa)

## Iteration

In [ ]:
fig, ax = plt.subplots()
contourf_p = ax.contourf(phi, th, Pres.T * pb / 100, levels=10)
ax.set_xlim(-30, 30)
# ax.set_ylim(1000, 600)
ax.set_xlabel(r"Latitude $\phi$ [°]")
ax.set_ylabel(r"Pressure $p$ [hPa]")
fig.colorbar(contourf_p, label=r"Potential temperature $\theta$ [K]")
plt.show()

nmax = 10  # maximum number of iteration sweeps
niter = 1  # number of Newton iterations per sweep
nup = 10  # print out the residual norm every nup iterations

u = np.zeros((2 * J - 1, K + 1))  # (-jc+1:jc-1,0:kc)
d = np.zeros((2 * J - 1, K + 1))  # (-jc+1:jc-1,0:kc)
g = np.zeros((2 * J - 1, K + 1))  # (-jc+1:jc-1,0:kc)
l = np.zeros((2 * J - 1, K + 1))  # (-jc+1:jc-1,0:kc)

n = 0
while n < nmax:
    n = n + 1
    rn = 0
    # Zebra relaxation in horizontal direction
    # first relax even horizontal lines, then odd lines
    for ks in range(2):
        iter = 0
        while iter < niter:
            iter = iter + 1
            for k in range(ks, K + 1, 2):
                kx = k + 1

                # fill in matrix elements for even points
                # point adjacent to south pole
                # using sin(phi)=sin(Phi)=-1 for boundary condition at south pole
                dxds = X[1, kx] + 1
                dxdz2 = X[0, kx + 1] - 2 * X[0, kx] + X[0, kx - 1]
                f2 = X[1, kx] - 1
                f3 = 1 - (f2 / 2) ** 2
                f4 = ((X[1, kx + 1] - X[1, kx - 1]) / 4) ** 2
                term2 = f1[0 + 1] * f4 * (1 + 3 * (f2 / 2) ** 2) / (f3**3)
                u[0, k] = dxdz2 + term2
                d[0, k] = -2 * dxds
                g[0, k] = -(
                    dxds * dxdz2
                    + f1[0 + 1] * f2 * f4 / f3**2
                    + gamma[0, k] * sigma[0, k] * fac2
                )
                # point adjacent to north pole
                # using sin(phi)=sin(Phi)=1 for boundary condition at south pole
                dxds = 1 - X[-2, kx]
                dxdz2 = X[-1, kx + 1] - 2 * X[-1, kx] + X[-1, kx - 1]
                f2 = 1 + X[-2, kx]
                f3 = 1 - (f2 / 2) ** 2
                f4 = ((X[-2, kx + 1] - X[-2, kx - 1]) / 4) ** 2
                term2 = f1[-1 - 1] * f4 * (1 + 3 * (f2 / 2) ** 2) / (f3**3)
                d[-1, k] = -2 * dxds
                l[-1, k] = -dxdz2 + term2
                g[-1, k] = -(
                    dxds * dxdz2
                    + f1[-1 - 1] * f2 * f4 / f3**2
                    + gamma[-1, k] * sigma[-1, k] * fac2
                )
                for j in range(2, 2 * J - 2, 2):
                    dxds = X[j + 1, kx] - X[j - 1, kx]
                    dxdz2 = X[j, kx + 1] - 2 * X[j, kx] + X[j, kx - 1]
                    f2 = X[j + 1, kx] + X[j - 1, kx]
                    f3 = 1 - (f2 / 2) ** 2
                    f4 = (
                        (
                            X[j + 1, kx + 1]
                            + X[j - 1, kx + 1]
                            - X[j + 1, kx - 1]
                            - X[j - 1, kx - 1]
                        )
                        / 4
                    ) ** 2
                    term2 = f1[j + 1] * f4 * (1 + 3 * (f2 / 2) ** 2) / (f3**3)
                    u[j, k] = dxdz2 + term2
                    d[j, k] = -2 * dxds
                    l[j, k] = -dxdz2 + term2
                    g[j, k] = -(
                        dxds * dxdz2
                        + f1[j + 1] * f2 * f4 / (f3**2)
                        + gamma[j, k] * sigma[j, k] * fac2
                    )
                # fill in matrix elements for odd points
                for j in range(1, 2 * J - 1, 2):
                    xjs = X[j, kx] * X[j, kx]
                    fj = xjs - scs[j + 1]
                    dj = 1 - xjs
                    l[j, k] = -1
                    d[j, k] = fac1[j + 1] * (
                        2 * X[j, kx] * (1 - scs[j + 1]) / (dj**2)
                    )
                    u[j, k] = 1
                    g[j, k] = -(fac1[j + 1] * (fj / dj) + X[j + 1, kx] - X[j - 1, kx])
            if iter == 1 & n == 1:
                plt.contourf(u.T)
                plt.colorbar()
                plt.show()
                plt.contourf(l.T)
                plt.colorbar()
                plt.show()
                plt.contourf(d.T)
                plt.colorbar()
                plt.show()
                plt.contourf(g.T)
                plt.colorbar()
                plt.show()

            u, d, l = gtdmsf(l, d, u)
            g = gtdmss(l, d, u, g)

            X[:, ks + 1 : K + 2 : 2] = X[:, ks + 1 : K + 2 : 2] + g[:, ks : K + 1 : 2]

        # update boundary
        X[::2, -1], X[::2, 0] = updateboundary.updateBoundary_M(
            X, scs, DZ, tdz, alpha, beta, eps, kappa
        )
        X[1::2, -1], X[1::2, 0] = updateboundary.s_boundary(X, fac1, scs)
        # Find gamma for general case
        pres[1:-1:2, :], gamma[1:-1:2, :], avep, dif = updateboundary.gamma_correction(
            pres, gamma, X, K, refp, kappa, alpha, tdz
        )
        # compute a corrected m field
        X[::2, :] = updateboundary.m_correction(
            X, pres, J, K, DZ, tdz, alpha, beta, eps, kappa, scs
        )

fig, ax = plt.subplots()
contourf_p = ax.contourf(phi, th, pres.T * pb / 100, levels=10)
ax.set_xlim(-30, 30)
# ax.set_ylim(1000, 600)
ax.set_xlabel(r"Latitude $\phi$ [°]")
ax.set_ylabel(r"Pressure $p$ [hPa]")
fig.colorbar(contourf_p, label=r"Potential temperature $\theta$ [K]")
plt.show()